<a href="https://www.kaggle.com/code/adinathjagtap777/titanic-machine-learning-from-disaster?scriptVersionId=288384507" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Cell 1: Setup environment
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#imports
import numpy as np 
import pandas as pd

In [ ]:
# Cell 2: Import libraries
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import tensorflow_decision_forests as tfdf

In [ ]:
# Cell 3: Load data
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
# Cell 4: Preprocessing
def preprocess(df):
    df = df.copy()
    df["Name"] = df["Name"].apply(lambda x: " ".join([v.strip(",()[].\"'") for v in x.split(" ")]))
    df["Ticket_number"] = df["Ticket"].apply(lambda x: x.split(" ")[-1])
    df["Ticket_item"] = df["Ticket"].apply(lambda x: "NONE" if len(x.split(" ")) == 1 else "_".join(x.split(" ")[0:-1]))
    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

In [ ]:
# Cell 5: Feature selection
input_features = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']

In [ ]:
# Cell 6: Create datasets

def tokenize_names(features, labels=None):
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels


import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)
    serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)


In [ ]:
# Cell 7: Train ensemble
predictions = None
num_predictions = 0

for i in range(100):
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1


predictions /= num_predictions

In [ ]:
# Cell 8: Create submission
submission = pd.DataFrame({
    "PassengerId": serving_df["PassengerId"],
    "Survived": (predictions >= 0.5).astype(int)
})

submission.to_csv("/kaggle/working/submission.csv", index=False)
print("Done")